## 第四章 常用的查询技巧

本次主要介绍的有视图、子查询、常见函数、谓词、case 等经常与select联合使用的技巧。

### 1.视图

#### 1.1 视图的基本概念

视图是依据SELECT语句来创建的一个虚拟的表(不同于直接操作数据表)，然后在这张虚拟表上做 SQL 操作。

视图与表的区别：**是否保存了实际的数据**，顺口溜：**视图不是表，视图是虚表，视图依赖于表**

视图是基于真实表的一张虚拟的表，其数据来源均建立在真实表的基础上。

![jupyter](https://oss.linklearner.com/wonderful-sql/ch03/ch03.01view.png)

视图的作用：
- 通过定义视图可以将频繁使用的SELECT语句保存以提高效率。
- 通过定义视图可以使用户看到的数据更加清晰。
- 通过定义视图可以不对外公开数据表全部字段，增强数据的保密性。
- 通过定义视图可以降低数据的冗余。

#### 1.2 创建视图

```sql
CREATE VIEW <视图名称>(<列名1>,<列名2>,...) AS <SELECT语句>
```

其中:
- SELECT 语句需要书写在 AS 关键字之后。 
- SELECT 语句中列的排列顺序和视图中列的排列顺序相同， SELECT 语句中的第 1 列就是视图中的第 1 列， SELECT 语句中的第 2 列就是视图中的第 2 列，以此类推。
- 视图的列名是在视图名称之后的列表中定义的。
- 视图名在数据库中需要是唯一的，不能与其他视图和表重名。
- 视图不仅可以基于真实表，我们也可以在视图的基础上继续创建视图。
- 在一般的DBMS中定义视图时不能使用ORDER BY语句,这是因为视图和表一样，**数据行都是没有顺序的**。

#### 1.3 基于单表的视图（示例）

```sql
CREATE VIEW productsum (product_type, cnt_product)
AS
SELECT product_type, COUNT(*)
  FROM product
 GROUP BY product_type
```

#### 1.4 基于多表的视图（示例）

```sql
CREATE VIEW view_shop_product(product_type, sale_price, shop_name)
AS
SELECT product_type, sale_price, shop_name
  FROM product,
       shop_product
 WHERE product.product_id = shop_product.product_id;
```

#### 1.5 视图上的基本查询（示例）

```sql
SELECT sale_price, shop_name
  FROM view_shop_product
 WHERE product_type = '衣服';
```

#### 1.6 修改视图结构
```sql
ALTER VIEW <视图名> AS <SELECT语句>

-- 示例
ALTER VIEW productSum
    AS
        SELECT product_type, sale_price
          FROM Product
         WHERE regist_da
```

#### 1.7 更新视图内容

因为视图是一个虚拟表，所以对视图的操作就是对底层基础表的操作，所以**在修改时只有满足底层基本表的定义才能成功修改**。

对于一个视图来说，如果包含以下结构的任意一种都是不可以被更新的：
- 聚合函数 SUM()、MIN()、MAX()、COUNT() 等。
- DISTINCT 关键字。
- GROUP BY 子句。
- HAVING 子句。
- UNION 或 UNION ALL 运算符。
- FROM 子句中包含多个表。
视图归根结底还是从表派生出来的，因此，**如果原表可以更新，那么视图中的数据也可以更新。反之亦然，如果视图发生了改变，而原表没有进行相应更新的话，就无法保证数据的一致性了**。

```sql
UPDATE productsum
   SET sale_price = '5000'
 WHERE product_type = '办公用品';
```

#### 1.8 删除视图

```sql
DROP VIEW <视图名1> [ , <视图名2> …]

-- 视图
DROP VIEW productSum; 
```


### 2.子查询

#### 2.1 子查询概念

子查询指一个查询语句嵌套在另一个查询语句内部的查询，在 SELECT 子句中先计算子查询，子查询结果作为外层另一个查询的过滤条件，查询可以基于一个表或者多个表。

子查询就是将用来定义视图的 SELECT 语句直接用于 FROM 子句当中。其中AS studentSum可以看作是子查询的名称，而且由于子查询是一次性的，所以子查询不会像视图那样保存在存储介质中， 而是在 SELECT 语句执行之后就消失了。

```sql
SELECT stu_name
FROM (
         SELECT stu_name, COUNT(*) AS stu_cnt
          FROM students_info
          GROUP BY stu_age) AS studentSum;
```
